## LIBRERIAS y CONSTANTES

In [1]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
#from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat, TreeFormat_all

from functions.aux import MetricTotalAccuraccy
# Load tree transformation
from functions.transform_inference import TransformerInference

from net import NeoNet

class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)


IS CUDA AVAILABLE: True


131072

### Basic parameters

In [2]:
URL = "/media/data/ccarmona/memoria/dataset/"
csv_name = 'new_dataset_7.2_subqueries'
model_name = 'Model_Adam'
model_path = "./" + model_name + '.pt'
x = [True,False]
active_new_data = x[0]
symbol = "ᶲ"
optimizer = "Adam"
#Este parametro sirve para elegir cierta cantidad de data ordenado por rangos de tiempo obtenidos.
## Entre más bajo menos data se seleccionara. Si es muy alto se tendran demasiados valores outliners, 
## pero si es muy bajo podría tenerse una data no representativa y se aumenta el riesgo de overfitting.
## Por otro lado min_data, simplemente da el valor minimo de tiempo de ejecución que tiene una consulta tomada
## en cuenta para hacer el modelo
#percent_of_data_or = 0.93
#min_time_or = 15
#max_time_or = 80
#percent_of_data = 1
#min_time = 5
#max_time = 80


In [3]:
epsilon = 0.75 ## EPSILON
lr_rl = 0.9 ## LEARNING RATE RL
disc_fac = 0.75 # DISCOUNT FACTOR RL 
epoch_rl = 200 ### TOTAL EPOCH POR DATA RL
n = "_" + model_name + "_gamma_" + str(disc_fac) + "_"##SALT Nombre archivos

### Features

In [4]:
# Columns to use.
list_columns = ['total_bgps', 'triples', 'treesize', 'join', 'left_join']

In [5]:
ds_train = pd.read_csv(URL + csv_name + '_ds_train_ft.csv', engine='python', encoding='utf-8')
ds_val = pd.read_csv(URL + csv_name + '_ds_val_ft.csv', engine='python', encoding='utf-8')
ds_test = pd.read_csv(URL + csv_name + '_ds_test_ft.csv', engine='python', encoding='utf-8')
ds_rl = pd.read_csv(URL + csv_name + '_ds_rl_ft.csv', engine='python', encoding='utf-8')

In [6]:
print("ds_train.shape",ds_train.shape)
print("ds_val.shape",ds_val.shape)
print("ds_test.shape",ds_test.shape)
print("ds_rl.shape",ds_rl.shape)

ds_train.shape (7938, 15)
ds_val.shape (1641, 15)
ds_test.shape (1072, 15)
ds_rl.shape (2000, 67)


# FUNCTIONS

In [7]:
def getpredictions_info(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate_with_card)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}

def getpredictions_info_nojc(x_val_tree, x_val_query, y_val):
    """
    Get statistics by a set of data. Need the previous trained model(availablre  form reg object).
    :param x_val_tree: Plan level features.
    :param x_val_query: Query level features.
    :param y_val: Real execution time
    :return: Dict with predictions and metrics (mae, rmse, mse)
    """
    Xt, Xq, Yv = reg.json_loads(x_val_tree, x_val_query.values, y_val)
    Xt = [reg.fix_tree(x) for x in Xt]
    Xt = reg.tree_transform.transform(Xt)

    pairs_val = list(zip(list(zip(Xt, Xq)), Yv))
    dataset_val = DataLoader(pairs_val, batch_size=batch_size, num_workers=0, shuffle=False, collate_fn=reg.collate)
    results_val = reg.predict_best(dataset_val)
    y_pred_val, y_real_val = zip(*results_val)
    mseval = mean_squared_error(y_real_val, y_pred_val)
    maeval = mean_absolute_error(y_real_val, y_pred_val)
    rmseval = np.sqrt(mseval)
    return {"pred": y_pred_val, "real" : y_real_val, "mse": mseval, "mae": maeval, "rmse": rmseval, "history": reg.history}



def getmax(x):
    lista=  list(x.values())
    maximo = 0
    for el in lista:
        if (maximo < float(el)):
            maximo = float(el)
    return maximo

def pred2index_dict(x, pred_to_index, maxcardinality):
    """
    get histogram from cardinality features. the values is normalized using the max cardinality of predicate in dataset.
    :param x: Tree data from x row sample.
    :param pred_to_index: dict with predicates and their index.
    :param maxcardinality: Max cardiniality in the dataset.
    :return: dictionary with feature json_cardinality.
    """
    resp = {}
    x = json.loads(x)
    for el in x.keys():
        if el in pred_to_index:
            resp[pred_to_index[el]] = float(x[el])/maxcardinality
    return resp

def prepare_query_level_data(x_train_query, x_val_query, x_test_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])

    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)

    x_train_query =pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)

    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))

    
    
    return x_train_query, x_val_query, x_test_query

def prepare_query_level_only_json_cardinality(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    
    x_train_query = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query


def prepare_query_level_data_full(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
    #Scale x_query data.
    xqtrain = x_train_query.drop(columns=['json_cardinality'])
    xqval   = x_val_query.drop(columns=['json_cardinality'])
    xqtest   = x_test_query.drop(columns=['json_cardinality'])
    xqrl = x_rl_query.drop(columns=['json_cardinality'])
    
    scalerx = StandardScaler()
    x_train_scaled = scalerx.fit_transform(xqtrain)
    x_val_scaled = scalerx.transform(xqval)
    x_test_scaled = scalerx.transform(xqtest)
    x_rl_scaled = scalerx.transform(xqrl)
    
    
    x_train_query = pd.concat([pd.DataFrame(x_train_scaled, index=xqtrain.index, columns=xqtrain.columns),x_train_query[['json_cardinality']]], axis=1)
    x_val_query =  pd.concat([pd.DataFrame(x_val_scaled,   index=xqval.index, columns=xqval.columns),x_val_query[['json_cardinality']]], axis=1)
    x_test_query =  pd.concat([pd.DataFrame(x_test_scaled,   index=xqtest.index, columns=xqtest.columns),x_test_query[['json_cardinality']]], axis=1)
    x_rl_query = pd.concat([pd.DataFrame(x_rl_scaled,   index=xqrl.index, columns=xqrl.columns),x_rl_query[['json_cardinality']]], axis=1)
    
    
    x_train_query['json_cardinality'] = x_train_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(),maxcardinality))
    x_val_query['json_cardinality'] = x_val_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_test_query['json_cardinality'] = x_test_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    x_rl_query['json_cardinality'] = x_rl_query['json_cardinality'].apply(lambda x: pred2index_dict(x, reg.get_pred(), maxcardinality))
    
    
    return x_train_query, x_val_query, x_test_query, x_rl_query

def prepare_query_level_data_full_no_jc(x_train_query, x_val_query, x_test_query, x_rl_query):
    """ Apply StandardScaller to columns except for json_cardinality that need other proccess"""
    scalerx = StandardScaler()
    columns_train, index_train = x_train_query.columns, x_train_query.index
    columns_val, index_val = x_val_query.columns, x_val_query.index
    columns_test, index_test = x_test_query.columns, x_test_query.index
    columns_rl, index_rl = x_rl_query.columns, x_rl_query.index
    
    x_train_scaled = scalerx.fit_transform(x_train_query)
    x_val_scaled = scalerx.transform(x_val_query)
    x_test_scaled = scalerx.transform(x_test_query)
    x_rl_scaled = scalerx.transform(x_rl_query)
    
    x_train_query = pd.DataFrame(x_train_scaled,   index=index_train, columns=columns_test)
    x_val_query = pd.DataFrame(x_val_scaled,   index=index_val, columns=columns_val)
    x_test_query = pd.DataFrame(x_test_scaled,   index=index_test, columns=columns_test)
    x_rl_query = pd.DataFrame(x_rl_scaled,   index=index_rl, columns=columns_rl)
    return x_train_query, x_val_query, x_test_query, x_rl_query



### TreeConv Neural Net

In [8]:
folds_execution = {}
print("Size Train: {}, Val {}".format(ds_train.shape[0], ds_val.shape[0]))

Size Train: 7938, Val 1641


In [9]:
# get query level data
x_train_query = ds_train[list_columns]
x_val_query   = ds_val[list_columns]

x_train_query_str = ds_train['query']
x_val_query_str = ds_train['query']

# get plan level datba
x_train_tree = ds_train['trees'].values
x_val_tree = ds_val['trees'].values

y_train = ds_train['time'].values
y_val = ds_val['time'].values

x_test_tree = ds_test['trees'].values
y_test = ds_test['time'].values
x_test_query   = ds_test[list_columns]

x_rl_tree = ds_rl['trees'].values
y_rl = ds_rl['time'].values
x_rl_query   = ds_rl[list_columns]

In [10]:
print("---------SHAPES-----------")
print("----------CLEAN-----------")
print(f'shape ds_train: {ds_train.shape}')
print(f'shape ds_val  : {ds_val.shape}')
print(f'shape ds_test : {ds_test.shape}')
print(f'shape ds_rl : {ds_rl.shape}')
print("")
print("-----TRAIN AND VAL DATA-----")
print("----------x_query_data----------")
print(f'shape x_val_query  : {x_val_query.shape}')
print(f'shape x_train_query: {x_train_query.shape}')
print("----------x_plan_level_data----------")
print(f'shape x_val_tree  : {x_val_tree.shape}')
print(f'shape x_train_tree: {x_train_tree.shape}')
print("----------y_data------------")
print(f'shape y_val  : {y_val.shape}')
print(f'shape y_train: {y_train.shape}')
print("")
print("----------TEST DATA----------")
print(f'shape x_test_tree : {x_test_tree.shape}')
print(f'shape x_test_query: {x_test_query.shape}')
print(f'shape y_test      : {y_test.shape}')
print("-----------------------")
print("")
print("----------RL DATA----------")
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")


---------SHAPES-----------
----------CLEAN-----------
shape ds_train: (7938, 15)
shape ds_val  : (1641, 15)
shape ds_test : (1072, 15)
shape ds_rl : (2000, 67)

-----TRAIN AND VAL DATA-----
----------x_query_data----------
shape x_val_query  : (1641, 5)
shape x_train_query: (7938, 5)
----------x_plan_level_data----------
shape x_val_tree  : (1641,)
shape x_train_tree: (7938,)
----------y_data------------
shape y_val  : (1641,)
shape y_train: (7938,)

----------TEST DATA----------
shape x_test_tree : (1072,)
shape x_test_query: (1072, 5)
shape y_test      : (1072,)
-----------------------

----------RL DATA----------
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


In [11]:
maxcardinality = 0
#maxcardinality =  x_train_query['json_cardinality'].apply(lambda x: json.loads(x)).apply(lambda x: getmax(x)).max()
maxcardinality

0

In [12]:
data_inference = {}
bao_base = ""
with (open(bao_base + "preprocess_inference_info_" + model_name + ".pickle", "rb")) as openfile:
    while True:
        try:
            data_inference = pickle.load(openfile)
        except EOFError:
            break
# Load Scaler for query features.
scalerx = None
with (open(bao_base + "scalerx_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            scalerx = pickle.load(openfile)
        except EOFError:
            break

pipeline_inverse = None
with (open(bao_base + "pipeline_inverse_" + model_name + ".pkl", "rb")) as openfile:
    while True:
        try:
            pipeline_inverse = pickle.load(openfile)
        except EOFError:
            break
with open('NeoNetValues_' + model_name +'.txt', 'r') as file:
    data = file.read().rstrip()
neo_data = ast.literal_eval(data)

IS CUDA AVAILABLE: True


In [13]:
def ActivationFuncReluOrLeaky(string):
    if string == "<class 'torch.nn.modules.activation.LeakyReLU'>":
        return nn.LeakyReLU
    elif string == "<class 'torch.nn.modules.activation.ReLU'>":
        return nn.ReLU
io_dim_model_data = int(neo_data['io_dim'])
query_input_size = int(neo_data['query_input_size'])
query_hidden_inputs = ast.literal_eval(neo_data['query_hidden_inputs'])
query_output = int(neo_data['query_output'])
tree_units = ast.literal_eval(neo_data['tree_units'])
tree_units_dense = ast.literal_eval(neo_data['tree_units_dense'])
tree_activation_tree = ActivationFuncReluOrLeaky(neo_data['activation_tree'])
tree_activation_dense = ActivationFuncReluOrLeaky(neo_data['activation_dense'])

def get_prediction_dataset_from_load(ds, list_columns):
    x_tree = ds["trees"].values
    x_query = ds[list_columns]
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    targets = list(ds["time"].values)
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}

def get_prediction_from_load(x_tree, x_query, targets):
    dataset_val = transformer.prepare(x_tree, x_query)
    predictions, predictions_extend = transformer.getpredictions_info(dataset_val)
    predictions_extend_full = [i[0] for i in predictions_extend]
    mseval = mean_squared_error(targets, predictions_extend_full)
    maeval = mean_absolute_error(targets, predictions_extend_full)
    rmseval = np.sqrt(mseval)
    return {"pred": predictions_extend_full, "real" : targets, "mse": mseval, "mae": maeval, "rmse": rmseval}


In [14]:
reg = NeoNet(
    io_dim_model_data,
    query_input_size,
    query_hidden_inputs,
    query_output=query_output,
    tree_units=tree_units,
    tree_units_dense=tree_units_dense,
    activation_tree=tree_activation_tree,
    activation_dense=tree_activation_dense,
)
device = torch.device("cuda")
# Loading model parameters.
reg_path = model_path
#reg.load_state_dict(torch.load(reg_path, map_location=torch.device('cpu')))
reg.load_state_dict(torch.load(reg_path))
reg.to(device)
transformer = TransformerInference(
        model=reg,
        tree_transform=data_inference["tree_transform"],
        pipeline_inverse=pipeline_inverse,
        scalerx=scalerx,
        maxcardinality=data_inference["maxcardinality"]
    )
print("Modelos Listos")

Activation function in treeConv layers: <class 'torch.nn.modules.activation.LeakyReLU'>
Activation function in tree dense output layers: <class 'torch.nn.modules.activation.ReLU'>
[BinaryTreeConvWithQData(
  (weights): Conv1d(1353, 512, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(512, 256, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), BinaryTreeConv(
  (weights): Conv1d(256, 128, kernel_size=(3,), stride=(3,))
), TreeLayerNorm(), TreeActivation(
  (activation): LeakyReLU(negative_slope=0.01)
), DynamicPooling(), Linear(in_features=128, out_features=64, bias=True), ReLU(), Linear(in_features=64, out_features=32, bias=True), ReLU(), Linear(in_features=32, out_features=1, bias=True)]
Modelos Listos


In [15]:
 ds_test['time']

0         1.687920
1         1.678266
2         1.109980
3         1.858232
4         1.598426
           ...    
1067    107.000000
1068    101.000000
1069    123.000000
1070    120.000000
1071    138.003726
Name: time, Length: 1072, dtype: float64

In [16]:
results = get_prediction_from_load(ds_test['trees'].values, ds_test[list_columns], ds_test['time'])

In [17]:
results

{'pred': [12.682406425476074,
  4.89743709564209,
  14.758463859558105,
  14.758463859558105,
  17.954404830932617,
  1.5937821865081787,
  14.758463859558105,
  1.5909326076507568,
  37.5489501953125,
  14.758463859558105,
  14.758463859558105,
  53.39189147949219,
  2.7885539531707764,
  14.758463859558105,
  14.758463859558105,
  14.758463859558105,
  44.66284942626953,
  4.89743709564209,
  1.6255521774291992,
  1.4945485591888428,
  14.758463859558105,
  1.5937821865081787,
  27.83724021911621,
  1.5922574996948242,
  36.944313049316406,
  13.64535903930664,
  2.7949717044830322,
  1.6255521774291992,
  36.944313049316406,
  14.758463859558105,
  14.758463859558105,
  84.42827606201172,
  1.5922574996948242,
  14.758463859558105,
  3.980473518371582,
  14.758463859558105,
  13.635659217834473,
  10.707067489624023,
  2.7885539531707764,
  4.35945987701416,
  6.078582763671875,
  39.093868255615234,
  2.813159227371216,
  14.754579544067383,
  24.007583618164062,
  14.7546920776367

## Reinforcement Learning

In [18]:
def GetTriplesSubtree(subtree_as_str):
    code_tpf = ['VAR_VAR_VAR', 'VAR_VAR_URI', 'VAR_URI_VAR', 'VAR_URI_URI', 'VAR_URI_LITERAL', 'VAR_VAR_LITERAL',
                'URI_URI_LITERAL', 'URI_URI_VAR', 'URI_URI_URI', 'URI_VAR_VAR', 'URI_VAR_URI', 'URI_VAR_LITERAL',
                'LITERAL_URI_VAR', 'LITERAL_URI_URI', 'LITERAL_URI_LITERAL']
    total_triples = 0
    for i in code_tpf:
        total_triples += subtree_as_str.count(i)
    #if subtree_as_str in code_tpf:
    #    total_triples += 1
    return total_triples

def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

def GetAllJoins(subtree_as_str):
    join = subtree_as_str.count('JOIN')
    left_join = subtree_as_str.count('LEFT_JOIN')
    return join-left_join, left_join

def GetIter(subtree_as_str):
    if "iter" in subtree_as_str:
        return 1
    else:
        return 0
    
def GetTotalBgp(state):
    bgp_list = []
    for s in state:
        bgp_list.append(s[1])
    bgp_list = set(bgp_list)
    return len(bgp_list)

def GetDataframe(subtree, state, columns):
    subtree_str = str(subtree)
    subtree_list = ast.literal_eval(subtree[0].tolist())
    total_bgps = GetTotalBgp(state)
    treesize = GetTreeSize(subtree_list, 1)
    triples = GetTriplesSubtree(subtree_str)
    join, left_join = GetAllJoins(subtree_str)
    iters = GetIter(subtree_str)
    
    values = [total_bgps, triples, treesize, join, left_join]
    x_rl_query = pd.DataFrame([values], columns = columns)
    return x_rl_query
    #print(x_rl_query)
    #scalerx = StandardScaler()
    #values_scaled = scalerx.fit_transform(x_rl_query)
    #x_rl_query = pd.DataFrame(values_scaled, columns = columns)
    #print(x_rl_query)
    
def RL_Actions(bgp):
    actions = []
    idx = 0
    for k,v in bgp.items():
        bgp_list = v['bgp_list']
        for b in v['bgp_list']:
            actions.append((idx,k,b['P'],b['triple_type']))
            idx += 1
    return actions


def RL_Initial_Step(actions):
    initial_state = []
    random_index = random.randint(0, len(actions)-1)
    random_action = actions[random_index]
    initial_state.append(random_action)
    return initial_state

def RL_available_actions(actions, current_state):
    if not current_state:
        return actions
    available_actions = sorted(list(set(actions) - set(current_state)))
    same_bgp_actions = []
    other_bgp_actions = []
    for i in available_actions:
        if current_state[-1][1] == i[1]:
            same_bgp_actions.append(i)
        else:
            other_bgp_actions.append(i)
    if same_bgp_actions:
        available_actions = same_bgp_actions
    else:
        available_actions = other_bgp_actions
    return available_actions


def RL_Argmax(array):
    argmax_list = np.argwhere(array == np.amax(array))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    return int(random.choice(argmax_list_flatten))

def RL_Argmax_available(q_value,available_actions):
    available_idx = [i[0] for i in available_actions]
    #print("available_actions",available_actions)
    #print("available_idx",available_idx)
    q_value_available = [q_value[i] for i in available_idx]
    #print("q_values",q_value, type(q_value))
    #print("q_values_available",q_value_available, type(q_value_available))
    
    argmax_list = np.argwhere(q_value_available == np.amax(q_value_available))
    argmax_list_flatten = [item for sublist in argmax_list for item in sublist]
    #print("argmax_list",argmax_list, type(argmax_list))
    #print("argmax_list_flatten",argmax_list_flatten, type(argmax_list_flatten))
    value = int(random.choice(argmax_list_flatten))
    #print("value",value)
    
    #print("..........................")
    return value


def RL_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RL_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RL_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
    new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
    #tree_format = TreeFormat_all(new_dicto,symbol)
    tree_format = TreeFormat(new_dicto,symbol)
    #tree_format = ast.literal_eval(tree_format)
    new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
    x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
    pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
    reward -= pred['pred'][0]
    return reward, terminal

def RL_Rebuild_Dictionary(bgp, final_state):
    new_dicto = {}
    bgp_names = list(set([i[1] for i in final_state]))
    ### Keys
    for i in bgp_names:
        new_dicto[i] = {"bgp_list" : [], "opt" : bgp[i]['opt']}
    for i in final_state:
        new_dicto[i[1]]["bgp_list"].append({'P' : i[2], 'triple_type' : i[3]})

    return new_dicto


def RL_First_Policy(actions):
    actions_length = len(actions)
    return np.zeros((actions_length,actions_length))

def RL_bgp_format(ds_rl):
    bgps_rl = ds_rl['bgps'].values
    bgps_rl = [ast.literal_eval(bgp) for bgp in bgps_rl]
    return bgps_rl

def RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx):
    x_rl_tree_c = x_rl_tree.copy()
    x_rl_query_c = x_rl_query.copy()
    y_rl_c = y_rl.copy()
    bgps_rl_c = bgps_rl.copy()
    x_rl_query_c = x_rl_query_c.reset_index(drop=True)
    columns = x_rl_query_c.columns
    values = x_rl_query_c.values[idx]    
    x_rl_tree_test = np.array([x_rl_tree_c[idx]])
    x_rl_query_test = pd.DataFrame([values],columns=columns)
    y_rl_test = np.array([y_rl_c[idx]])
    bgps_test = bgps_rl_c[idx]
    return x_rl_tree_test, x_rl_query_test, y_rl_test, bgps_test


def RL_get_final_state_bgp_tree(q_value,actions,bgp,symbol):
    q_value = q_value.tolist()
    best_state = []
    arg_max = RL_Argmax(q_value[0])
    best_state.append(actions[arg_max])
        
    for q_val in q_value[1:]:
        available_actions = RL_available_actions(actions, best_state)
        chosen_action_available_idx = RL_Argmax_available(q_val,available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        best_state.append(chosen_action) 
    new_dicto = RL_Rebuild_Dictionary(bgp, best_state)
    #new_tree = np.array([str(TreeFormat_all(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    new_tree = np.array([str(TreeFormat(new_dicto,symbol)).replace('"', ';').replace("'", '"')])
    
    return best_state, new_dicto, new_tree


def RL_results_functions(idx,pred_old, pred_new,prl_tol1,prl_tol2):
    difference = pred_new - pred_old
    abs_diff = np.abs(difference)
    best_of_the_pred = []
    similar_of_the_pred = []
    worst_of_the_pred = []
    if abs_diff < prl_tol1:
        best_of_the_pred.append(idx)
    elif abs_diff < prl_tol2:
        similar_of_the_pred.append(idx)
    else:
        worst_of_the_pred.append(idx)
    return best_of_the_pred, similar_of_the_pred, worst_of_the_pred
    
def RLNeo_Next_step(actions,
                current_state,
                reward,
                q_value,
                x_rl_tree_ind,
                x_rl_query_ind,
                y_rl_ind,
                bgp_ind,
                epsilon
                ):
    random_num = np.random.random()
    available_actions = RL_available_actions(actions, current_state)
    actual_state = len(current_state) - 1
    
    if random_num < epsilon:
        chosen_action_available_idx = random.randint(0, len(available_actions)-1)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        #print(chosen_action_idx)
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    else:
        #print(current_state)
        chosen_action_available_idx = RL_Argmax_available(q_value[actual_state],available_actions)
        chosen_action = available_actions[chosen_action_available_idx]
        chosen_action_idx = chosen_action[0]
        current_state.append(chosen_action)
        reward, terminal = RLNeo_Reward(actions,
                                     available_actions,
                                     current_state,
                                     chosen_action,
                                     reward,
                                     x_rl_tree_ind,
                                     x_rl_query_ind,
                                     y_rl_ind,
                                     bgp_ind)
    return current_state, reward, terminal, chosen_action, chosen_action_idx

def RLNeo_Reward(actions,
              available_actions,
              current_state,
              chosen_action,
              reward,
              x_rl_tree_ind,
              x_rl_query_ind,
              y_rl_ind,
              bgp_ind):
    terminal = False
    reward = 0
    #print(available_actions)
    #if chosen_action not in available_actions:
    #    reward -= 100000
    if len(current_state) == len(actions):
        terminal = True
        new_dicto = RL_Rebuild_Dictionary(bgp_ind, current_state)
        tree_format = TreeFormat(new_dicto,symbol)
        #tree_format = ast.literal_eval(tree_format)
        new_tree = np.array([str(tree_format).replace('"', ';').replace("'", '"')])
        x_rl_query = GetDataframe(new_tree, current_state, x_rl_query_ind.columns)
        pred = get_prediction_from_load(new_tree, x_rl_query, y_rl_ind)
        reward -= pred['pred'][0]
    return reward, terminal
    
    
def RLNeo_q_values(q_value_neo,current_state, reward):
    for q_val, act in zip(q_value_neo,current_state):
        if q_val[act[0]] == 0:
            q_val[act[0]] = reward
        else:
            q_val[act[0]] = max(reward,q_val[act[0]])
    return q_value_neo

def RL_max_available_actions(q_value, old_states, actions):
    available_actions = RL_available_actions(actions, old_states)
    index_av = [i[0] for i in available_actions]
    q_value_acotado = q_value[index_av[0]:index_av[-1]+1]
    return np.max(q_value_acotado)

In [19]:
print("----------RL DATA----------")
print(f'shape ds_rl : {ds_rl.shape}')
print(f'shape x_rl_tree : {x_rl_tree.shape}')
print(f'shape x_rl_query: {x_rl_query.shape}')
print(f'shape y_rl      : {y_rl.shape}')
print("-----------------------")
bgps_rl = RL_bgp_format(ds_rl)

----------RL DATA----------
shape ds_rl : (2000, 67)
shape x_rl_tree : (2000,)
shape x_rl_query: (2000, 5)
shape y_rl      : (2000,)
-----------------------


## MULTI-VAL

##### Seleccionar data

In [20]:
min_num_actions = -1
buenos_idx = []
aaa = []
for i in range(0,x_rl_tree.shape[0]):
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,i)
    actions = RL_Actions(bgp_ind)
    if len(actions) >= min_num_actions:
        aaa.append(y_rl_ind)
        buenos_idx.append(i)
total_buenos_idx = len(buenos_idx)
print("total_idx",total_buenos_idx)
rl_columns = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree',
            #'history_old',
            #'history_new'
             ]
 
iteration = [
            'index',
            'iteration',
            'pred_old',
            'pred_new',
            'real_old',
            'real_new',
            'mse_old',
            'mse_new',
            'mae_old',
            'mae_new',
            'rmse_old',
            'rmse_new',
            'old_tree',
            'new_tree'
             ]

total_idx 2000


##### Parameters

In [21]:
#epsilon = 0.7 ## EPSILON
#lr_rl = 0.9 ## LEARNING RATE
#disc_fac = 0.999 ## DISCOUNT FACTOR
#epoch_rl = 200 ### TOTAL EPOCH POR DATA
#n = model_name ##SALT Nombre archivos

### Q - Learning - multival

##### Parameters

In [22]:
epsilon_ql = epsilon ## EPSILON
lr_ql = lr_rl ## LEARNING RATE
disc_fac_ql = disc_fac ## DISCOUNT FACTOR
epoch_ql = epoch_rl ### TOTAL EPOCH POR DATA

##### Loop

In [23]:
values_ql = []
values_final_ql = []
c_ql = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_ql = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_ql % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_ql}/{total_buenos_idx}, index {idx} evaluating')
    ####################################Q LEARNING###############################################
    for i in range(0,epoch_ql):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,   
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_ql,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_ql)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_ql,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_ql)
            
            new_state = len(current_state)-1
            old_q_value = q_value_ql[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_ql[new_state,:], old_states_val, actions)
            temporal_difference = reward + disc_fac_ql * q_max - old_q_value
            #temporal_difference = reward + disc_fac_ql * np.max(q_value_ql[new_state,:]) - old_q_value
            new_q_value = old_q_value + lr_ql * temporal_difference
            q_value_ql[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_ql[old_state,prev_action]
        new_q_value = old_q_value + lr_ql * (reward - old_q_value)
        q_value_ql[old_state,prev_action] = new_q_value
        ################################################################################################
        
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_ql,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_ql.append(row)
    values_final_ql.append(row)
    c_ql += 1
print("len(values_ql[0])", len(values_ql[0]))
print("len(values_ql)", len(values_ql))

Start: 2022-01-04 04:54:00.107911
2022-01-04 04:54:46.408349 : 10/2000, index 9 evaluating
2022-01-04 04:55:43.323246 : 20/2000, index 19 evaluating
2022-01-04 04:56:39.425605 : 30/2000, index 29 evaluating
2022-01-04 04:57:33.977536 : 40/2000, index 39 evaluating
2022-01-04 04:58:30.293587 : 50/2000, index 49 evaluating
2022-01-04 04:59:30.864809 : 60/2000, index 59 evaluating
2022-01-04 05:00:16.771962 : 70/2000, index 69 evaluating
2022-01-04 05:01:18.971455 : 80/2000, index 79 evaluating
2022-01-04 05:02:09.240032 : 90/2000, index 89 evaluating
2022-01-04 05:03:04.010133 : 100/2000, index 99 evaluating
2022-01-04 05:04:09.879899 : 110/2000, index 109 evaluating
2022-01-04 05:05:14.184246 : 120/2000, index 119 evaluating
2022-01-04 05:06:11.111664 : 130/2000, index 129 evaluating
2022-01-04 05:07:03.008594 : 140/2000, index 139 evaluating
2022-01-04 05:07:59.737632 : 150/2000, index 149 evaluating
2022-01-04 05:08:51.581394 : 160/2000, index 159 evaluating
2022-01-04 05:09:54.475622

2022-01-04 07:03:39.414425 : 1370/2000, index 1369 evaluating
2022-01-04 07:04:41.469361 : 1380/2000, index 1379 evaluating
2022-01-04 07:05:41.929345 : 1390/2000, index 1389 evaluating
2022-01-04 07:06:55.249501 : 1400/2000, index 1399 evaluating
2022-01-04 07:08:02.147698 : 1410/2000, index 1409 evaluating
2022-01-04 07:08:56.409018 : 1420/2000, index 1419 evaluating
2022-01-04 07:09:55.441891 : 1430/2000, index 1429 evaluating
2022-01-04 07:10:57.618820 : 1440/2000, index 1439 evaluating
2022-01-04 07:11:56.734832 : 1450/2000, index 1449 evaluating
2022-01-04 07:13:23.003502 : 1460/2000, index 1459 evaluating
2022-01-04 07:14:25.184151 : 1470/2000, index 1469 evaluating
2022-01-04 07:15:30.220910 : 1480/2000, index 1479 evaluating
2022-01-04 07:16:32.617706 : 1490/2000, index 1489 evaluating
2022-01-04 07:17:33.318846 : 1500/2000, index 1499 evaluating
2022-01-04 07:18:32.657408 : 1510/2000, index 1509 evaluating
2022-01-04 07:19:32.014041 : 1520/2000, index 1519 evaluating
2022-01-

In [24]:
ql_df = pd.DataFrame(values_ql,columns=rl_columns)
ql_df_iteration = ql_df[iteration]
ql_df_iteration.to_csv(URL + 'rl_csvs/ql_df_iteration'+str(n)+'.csv',index=False)

In [25]:
ql_df_final = pd.DataFrame(values_final_ql,columns=rl_columns)
ql_df_final.to_csv(URL + 'rl_csvs/ql_df_final'+str(n)+'.csv',index=False)

In [26]:
ql_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.248253,75.542145,49.0,49.0,52.537170,704.485449,7.248253,26.542145,7.248253,26.542145,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.192539,78.139153,49.0,49.0,51.732620,849.090210,7.192539,29.139153,7.192539,29.139153,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
2,0,2,56.214851,83.670593,49.0,49.0,52.054080,1202.050037,7.214851,34.670593,7.214851,34.670593,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
3,0,3,56.249290,78.708252,49.0,49.0,52.552212,882.580234,7.249290,29.708252,7.249290,29.708252,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
4,0,4,56.259995,82.467873,49.0,49.0,52.707520,1120.098498,7.259995,33.467873,7.259995,33.467873,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,110.907471,30.538246,124.0,124.0,171.414323,8735.099432,13.092529,93.461754,13.092529,93.461754,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,111.201767,31.156700,124.0,124.0,163.794769,8619.878330,12.798233,92.843300,12.798233,92.843300,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,111.595375,31.006317,124.0,124.0,153.874720,8647.825052,12.404625,92.993683,12.404625,92.993683,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,106.745956,30.607718,124.0,124.0,297.702020,8722.118428,17.254044,93.392282,17.254044,93.392282,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [27]:
ql_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.232231,56.244297,49.0,49.0,52.305167,52.479839,7.232231,7.244297,7.232231,7.244297,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,57.225586,58.575989,46.0,46.0,126.013780,158.155494,11.225586,12.575989,11.225586,12.575989,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.495953,61.298717,48.0,48.0,156.148832,176.855887,12.495953,13.298717,12.495953,13.298717,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.020546,106.083488,49.0,49.0,3846.548121,3258.524655,62.020546,57.083488,62.020546,57.083488,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,137.705841,139.693100,111.0,111.0,713.201947,823.293986,26.705841,28.693100,26.705841,28.693100,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,103.346443,59.392326,122.0,122.0,347.955182,3919.720799,18.653557,62.607674,18.653557,62.607674,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.118225,73.939987,109.0,109.0,581.688782,1229.204499,24.118225,35.060013,24.118225,35.060013,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,157.323822,75.174332,136.0,136.0,454.705386,3699.761928,21.323822,60.825668,21.323822,60.825668,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## RL-Neo Multival

###### Parameters

In [28]:
epsilon_neo = epsilon ## EPSILON
epoch_neo = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [29]:
values_neo = []
values_final_neo = []
c_neo = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_neo = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_neo % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_neo}/{total_buenos_idx}, index {idx} evaluating')
    ############# ALGORITMO NEO #####################3
    for i in range(0,epoch_neo):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_neo,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_neo)    
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RLNeo_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_neo,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_neo)
        q_value_neo = RLNeo_q_values(q_value_neo,current_state, reward)
        ##############################################################
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_neo,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]
        values_neo.append(row)
    values_final_neo.append(row)
    c_neo += 1
#print("values_neo ", values_neo)
print("len(values_neo[0])",len(values_neo[0]) )
print("len(values_neo)", len(values_neo))

Start: 2022-01-04 08:11:56.913438
2022-01-04 08:12:34.556588 : 10/2000, index 9 evaluating
2022-01-04 08:13:16.525817 : 20/2000, index 19 evaluating
2022-01-04 08:13:58.447443 : 30/2000, index 29 evaluating
2022-01-04 08:14:40.243114 : 40/2000, index 39 evaluating
2022-01-04 08:15:22.119937 : 50/2000, index 49 evaluating
2022-01-04 08:16:04.247989 : 60/2000, index 59 evaluating
2022-01-04 08:16:45.514272 : 70/2000, index 69 evaluating
2022-01-04 08:17:27.659502 : 80/2000, index 79 evaluating
2022-01-04 08:18:07.326618 : 90/2000, index 89 evaluating
2022-01-04 08:18:48.655455 : 100/2000, index 99 evaluating
2022-01-04 08:19:29.751500 : 110/2000, index 109 evaluating
2022-01-04 08:20:12.281563 : 120/2000, index 119 evaluating
2022-01-04 08:20:54.337641 : 130/2000, index 129 evaluating
2022-01-04 08:21:36.077416 : 140/2000, index 139 evaluating
2022-01-04 08:22:18.015668 : 150/2000, index 149 evaluating
2022-01-04 08:22:59.740172 : 160/2000, index 159 evaluating
2022-01-04 08:23:42.123722

2022-01-04 09:47:54.243813 : 1370/2000, index 1369 evaluating
2022-01-04 09:48:36.603932 : 1380/2000, index 1379 evaluating
2022-01-04 09:49:18.860793 : 1390/2000, index 1389 evaluating
2022-01-04 09:50:01.895734 : 1400/2000, index 1399 evaluating
2022-01-04 09:50:44.433529 : 1410/2000, index 1409 evaluating
2022-01-04 09:51:26.414548 : 1420/2000, index 1419 evaluating
2022-01-04 09:52:08.625871 : 1430/2000, index 1429 evaluating
2022-01-04 09:52:51.161753 : 1440/2000, index 1439 evaluating
2022-01-04 09:53:33.381251 : 1450/2000, index 1449 evaluating
2022-01-04 09:54:17.286528 : 1460/2000, index 1459 evaluating
2022-01-04 09:54:59.717619 : 1470/2000, index 1469 evaluating
2022-01-04 09:55:42.327677 : 1480/2000, index 1479 evaluating
2022-01-04 09:56:24.747401 : 1490/2000, index 1489 evaluating
2022-01-04 09:57:07.089451 : 1500/2000, index 1499 evaluating
2022-01-04 09:57:49.297272 : 1510/2000, index 1509 evaluating
2022-01-04 09:58:31.441775 : 1520/2000, index 1519 evaluating
2022-01-

In [30]:
neo_df = pd.DataFrame(values_neo,columns=rl_columns)
neo_df_iteration = neo_df[iteration]
neo_df_iteration.to_csv(URL + 'rl_csvs/neo_df_iteration'+str(n)+'.csv',index=False)

In [31]:
neo_df_final = pd.DataFrame(values_final_neo,columns=rl_columns)
neo_df_final.to_csv(URL + 'rl_csvs/neo_df_final'+str(n)+'.csv',index=False)

In [32]:
neo_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.177513,56.256416,49.0,49.0,51.516695,52.655578,7.177513,7.256416,7.177513,7.256416,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,0,1,56.873852,56.235672,49.0,49.0,61.997542,52.354949,7.873852,7.235672,7.873852,7.235672,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.903801,56.198292,49.0,49.0,62.470070,51.815405,7.903801,7.198292,7.903801,7.198292,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.195538,56.272388,49.0,49.0,51.775761,52.887634,7.195538,7.272388,7.195538,7.272388,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.179066,56.190876,49.0,49.0,51.538984,51.708698,7.179066,7.190876,7.179066,7.190876,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.017822,30.928913,124.0,124.0,168.536939,8662.227214,12.982178,93.071087,12.982178,93.071087,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,111.230186,30.894903,124.0,124.0,163.068138,8668.559053,12.769814,93.105097,12.769814,93.105097,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,110.874710,30.896347,124.0,124.0,172.273235,8668.290193,13.125290,93.103653,13.125290,93.103653,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,105.826904,30.916302,124.0,124.0,330.261407,8664.574884,18.173096,93.083698,18.173096,93.083698,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [33]:
neo_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.286812,57.131954,49.0,49.0,53.097627,66.128679,7.286812,8.131954,7.286812,8.131954,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.680195,58.893009,46.0,46.0,160.787342,166.229686,12.680195,12.893009,12.680195,12.893009,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,59.894581,60.495953,48.0,48.0,141.481053,156.148832,11.894581,12.495953,11.894581,12.495953,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.650318,45.571594,48.0,48.0,5.521005,5.897155,2.349682,2.428406,2.349682,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.092361,93.161423,49.0,49.0,3855.461350,1950.231257,62.092361,44.161423,62.092361,44.161423,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P885;ᶲIRI_ID;...dire..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.713364,82.181847,111.0,111.0,824.457252,830.485964,28.713364,28.818153,28.713364,28.818153,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_VAR_V..."
1996,1996,199,100.297623,58.737194,122.0,122.0,470.993181,4002.182615,21.702377,63.262806,21.702377,63.262806,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,132.996262,73.272987,109.0,109.0,575.820571,1276.419432,23.996262,35.727013,23.996262,35.727013,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.150681,73.785843,136.0,136.0,490.652648,3870.601344,22.150681,62.214157,22.150681,62.214157,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Sarsa Multival

###### Parameters

In [34]:
epsilon_sar = epsilon ## EPSILON
lr_sar = lr_rl ## LEARNING RATE
disc_fac_sar = disc_fac ## DISCOUNT FACTOR
epoch_sar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [35]:
values_sar = []
values_final_sar = []
c_sar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_sar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_sar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_sar}/{total_buenos_idx}, index {idx} evaluating')
    ############################ALGORITMO SARSA##########################################
    for i in range(0,epoch_sar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_sar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_sar)
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_sar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_sar)

            new_state = len(current_state)-1
            old_q_value = q_value_sar[old_state,prev_action]
            temporal_difference = reward + (disc_fac_sar * q_value_sar[new_state,chosen_action_idx]) - old_q_value
            new_q_value = old_q_value + lr_sar * temporal_difference
            q_value_sar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        old_q_value = q_value_sar[old_state,prev_action]
        new_q_value = old_q_value + lr_sar * (reward - old_q_value)
        q_value_sar[old_state,prev_action] = new_q_value
        ###############################################################################################33
        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_sar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_sar.append(row)
    values_final_sar.append(row)
    c_sar += 1
#print("values_sar ", values_sar)
print("len(values_sar[0])",len(values_sar[0]) )
print("len(values_sar)", len(values_sar))

Start: 2022-01-04 10:32:58.596172
2022-01-04 10:33:52.594204 : 10/2000, index 9 evaluating
2022-01-04 10:34:53.657552 : 20/2000, index 19 evaluating
2022-01-04 10:35:53.589194 : 30/2000, index 29 evaluating
2022-01-04 10:36:51.399934 : 40/2000, index 39 evaluating
2022-01-04 10:37:50.836668 : 50/2000, index 49 evaluating
2022-01-04 10:38:55.024912 : 60/2000, index 59 evaluating
2022-01-04 10:39:41.997579 : 70/2000, index 69 evaluating
2022-01-04 10:40:45.770075 : 80/2000, index 79 evaluating
2022-01-04 10:41:39.216019 : 90/2000, index 89 evaluating
2022-01-04 10:42:37.384485 : 100/2000, index 99 evaluating
2022-01-04 10:43:46.538725 : 110/2000, index 109 evaluating
2022-01-04 10:44:54.875381 : 120/2000, index 119 evaluating
2022-01-04 10:45:54.860917 : 130/2000, index 129 evaluating
2022-01-04 10:46:49.745408 : 140/2000, index 139 evaluating
2022-01-04 10:47:49.201417 : 150/2000, index 149 evaluating
2022-01-04 10:48:44.009181 : 160/2000, index 159 evaluating
2022-01-04 10:49:49.949185

2022-01-04 12:48:30.715020 : 1370/2000, index 1369 evaluating
2022-01-04 12:49:33.653019 : 1380/2000, index 1379 evaluating
2022-01-04 12:50:34.727131 : 1390/2000, index 1389 evaluating
2022-01-04 12:51:48.436339 : 1400/2000, index 1399 evaluating
2022-01-04 12:52:56.038050 : 1410/2000, index 1409 evaluating
2022-01-04 12:53:51.118307 : 1420/2000, index 1419 evaluating
2022-01-04 12:54:50.953707 : 1430/2000, index 1429 evaluating
2022-01-04 12:55:54.256219 : 1440/2000, index 1439 evaluating
2022-01-04 12:56:53.978088 : 1450/2000, index 1449 evaluating
2022-01-04 12:58:20.935685 : 1460/2000, index 1459 evaluating
2022-01-04 12:59:23.710071 : 1470/2000, index 1469 evaluating
2022-01-04 13:00:29.567300 : 1480/2000, index 1479 evaluating
2022-01-04 13:01:32.486764 : 1490/2000, index 1489 evaluating
2022-01-04 13:02:33.618449 : 1500/2000, index 1499 evaluating
2022-01-04 13:03:33.512011 : 1510/2000, index 1509 evaluating
2022-01-04 13:04:33.415583 : 1520/2000, index 1519 evaluating
2022-01-

In [36]:
sarsa_df = pd.DataFrame(values_sar,columns=rl_columns)
sarsa_df_iteration = sarsa_df[iteration]
sarsa_df_iteration.to_csv(URL + 'rl_csvs/sarsa_df_iteration'+str(n)+'.csv',index=False)

In [37]:
sarsa_df_final = pd.DataFrame(values_final_sar,columns=rl_columns)
sarsa_df_final.to_csv(URL + 'rl_csvs/sarsa_df_final'+str(n)+'.csv',index=False)

In [38]:
sarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.221535,76.792679,49.0,49.0,52.150564,772.432997,7.221535,27.792679,7.221535,27.792679,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.177540,56.191067,49.0,49.0,51.517078,51.711441,7.177540,7.191067,7.177540,7.191067,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
2,0,2,56.205765,56.179287,49.0,49.0,51.923046,51.542161,7.205765,7.179287,7.205765,7.179287,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.249481,56.219490,49.0,49.0,52.554978,52.121037,7.249481,7.219490,7.249481,7.219490,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
4,0,4,56.217087,56.241486,49.0,49.0,52.086342,52.439114,7.217087,7.241486,7.217087,7.241486,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,110.297295,30.815771,124.0,124.0,187.764135,8683.300515,13.702705,93.184229,13.702705,93.184229,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,109.665955,30.908821,124.0,124.0,205.464858,8665.967588,14.334045,93.091179,14.334045,93.091179,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,110.887459,31.175045,124.0,124.0,171.938737,8616.472268,13.112541,92.824955,13.112541,92.824955,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,111.563705,29.629663,124.0,124.0,154.661422,8905.760417,12.436295,94.370337,12.436295,94.370337,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [39]:
sarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.309734,56.190029,49.0,49.0,53.432216,51.696519,7.309734,7.190029,7.309734,7.190029,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.936867,57.719482,46.0,46.0,167.362522,137.346268,12.936867,11.719482,12.936867,11.719482,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.337692,59.657524,48.0,48.0,152.218650,135.897868,12.337692,11.657524,12.337692,11.657524,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,110.833961,111.040688,49.0,49.0,3823.438793,3849.046913,61.833961,62.040688,61.833961,62.040688,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.656128,139.452454,111.0,111.0,821.173668,809.542117,28.656128,28.452454,28.656128,28.452454,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,102.440750,59.347446,122.0,122.0,382.564256,3925.342467,19.559250,62.652554,19.559250,62.652554,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,131.159790,72.397957,109.0,109.0,491.056295,1339.709563,22.159790,36.602043,22.159790,36.602043,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,158.698395,71.118034,136.0,136.0,515.217125,4209.669465,22.698395,64.881966,22.698395,64.881966,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


## Expected-Sarsa Multival

###### Parameters

In [40]:
epsilon_exsar = epsilon ## EPSILON
epsilon_exsar_alg = epsilon
lr_exsar = lr_rl ## LEARNING RATE
disc_fac_exsar = disc_fac ## DISCOUNT FACTOR
epoch_exsar = epoch_rl ### TOTAL EPOCH POR DATA

###### Loop

In [41]:
values_exsar = []
values_final_exsar = []
c_exsar = 1
print(f'Start: {str(datetime.datetime.now())}')
for idx in buenos_idx:
    x_rl_tree_ind, x_rl_query_ind, y_rl_ind, bgp_ind = RL_get_data(x_rl_tree, x_rl_query, y_rl, bgps_rl,idx)
    actions = RL_Actions(bgp_ind)
    num_actions = len(actions)
    q_value_exsar = RL_First_Policy(actions) ##INITIAL POLICITY
    if c_exsar % 10 == 0:
        print(f'{str(datetime.datetime.now())} : {c_exsar}/{total_buenos_idx}, index {idx} evaluating')
        
    ##### ALGORITMO EXPECTED SARSA
    for i in range(0,epoch_exsar):
        current_state = []
        reward = 0
        terminal = False
        current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                        current_state,
                                                                                        reward,
                                                                                        q_value_exsar,
                                                                                        x_rl_tree_ind,
                                                                                        x_rl_query_ind,
                                                                                        y_rl_ind,
                                                                                        bgp_ind,
                                                                                        epsilon_exsar)    
        old_state = len(current_state)-1
        prev_action = chosen_action_idx
        while not terminal:
            old_states_val = current_state.copy()
            current_state, reward, terminal, chosen_action, chosen_action_idx = RL_Next_step(actions,
                                                                                            current_state,
                                                                                            reward,
                                                                                            q_value_exsar,
                                                                                            x_rl_tree_ind,
                                                                                            x_rl_query_ind,
                                                                                            y_rl_ind,
                                                                                            bgp_ind,
                                                                                            epsilon_exsar)
            new_state = len(current_state)-1
            array_state = np.array(current_state)




            expected_q = 0
            old_q_value = q_value_exsar[old_state,prev_action]
            q_max = RL_max_available_actions(q_value_exsar[new_state,:], old_states_val, actions)
            #q_max = np.max(q_value_exsar[new_state,:])
            pi = np.ones(num_actions)*epsilon_exsar_alg/num_actions \
                + (q_value_exsar[new_state,:] == q_max)*(1-epsilon_exsar_alg)/np.sum(q_value_exsar[new_state,:] == q_max)
    
            expected_q = np.sum(q_value_exsar[new_state,:]*pi)
            temporal_difference = reward + disc_fac_exsar * expected_q - old_q_value
            new_q_value = old_q_value +  lr_exsar*temporal_difference
            q_value_exsar[old_state,prev_action] = new_q_value
            old_state = new_state
            prev_action = chosen_action_idx
        #########################################################
        old_q_value = q_value_exsar[old_state,prev_action]
        new_q_value = old_q_value + lr_exsar * (reward - old_q_value)
        q_value_exsar[old_state,prev_action] = new_q_value

        best_state, new_dicto, new_tree = RL_get_final_state_bgp_tree(q_value_exsar,actions,bgp_ind,symbol)
        rl_stats_old = get_prediction_from_load(x_rl_tree_ind, x_rl_query_ind, y_rl_ind)
        rl_stats_new = get_prediction_from_load(new_tree, x_rl_query_ind, y_rl_ind)
        
        row = [
                    idx,
                    i,
                    rl_stats_old['pred'][0],
                    rl_stats_new['pred'][0],
                    float(rl_stats_old['real'][0]),
                    float(rl_stats_new['real'][0]), 
                    rl_stats_old['mse'],
                    rl_stats_new['mse'],
                    rl_stats_old['mae'],
                    rl_stats_new['mae'],
                    rl_stats_old['rmse'],
                    rl_stats_new['rmse'],
                    str(x_rl_tree_ind),
                    str(new_tree),
           ]

        values_exsar.append(row)
    values_final_exsar.append(row)
    c_exsar += 1
#print("values_exsar ", values_exsar)
print("len(values_exsar[0])",len(values_exsar[0]) )
print("len(values_exsar)", len(values_exsar))

Start: 2022-01-04 13:57:27.425452
2022-01-04 13:58:21.631417 : 10/2000, index 9 evaluating
2022-01-04 13:59:22.657211 : 20/2000, index 19 evaluating
2022-01-04 14:00:22.886713 : 30/2000, index 29 evaluating
2022-01-04 14:01:20.989974 : 40/2000, index 39 evaluating
2022-01-04 14:02:20.625892 : 50/2000, index 49 evaluating
2022-01-04 14:03:23.762912 : 60/2000, index 59 evaluating
2022-01-04 14:04:12.486914 : 70/2000, index 69 evaluating
2022-01-04 14:05:17.018829 : 80/2000, index 79 evaluating
2022-01-04 14:06:10.195983 : 90/2000, index 89 evaluating
2022-01-04 14:07:07.957364 : 100/2000, index 99 evaluating
2022-01-04 14:08:16.662860 : 110/2000, index 109 evaluating
2022-01-04 14:09:25.328696 : 120/2000, index 119 evaluating
2022-01-04 14:10:25.563895 : 130/2000, index 129 evaluating
2022-01-04 14:11:20.572105 : 140/2000, index 139 evaluating
2022-01-04 14:12:20.096067 : 150/2000, index 149 evaluating
2022-01-04 14:13:15.055129 : 160/2000, index 159 evaluating
2022-01-04 14:14:21.668056

2022-01-04 16:13:26.546087 : 1370/2000, index 1369 evaluating
2022-01-04 16:14:29.394241 : 1380/2000, index 1379 evaluating
2022-01-04 16:15:30.734738 : 1390/2000, index 1389 evaluating
2022-01-04 16:16:44.999081 : 1400/2000, index 1399 evaluating
2022-01-04 16:17:52.721508 : 1410/2000, index 1409 evaluating
2022-01-04 16:18:47.685649 : 1420/2000, index 1419 evaluating
2022-01-04 16:19:47.451650 : 1430/2000, index 1429 evaluating
2022-01-04 16:20:50.466070 : 1440/2000, index 1439 evaluating
2022-01-04 16:21:50.075979 : 1450/2000, index 1449 evaluating
2022-01-04 16:23:17.874583 : 1460/2000, index 1459 evaluating
2022-01-04 16:24:20.628961 : 1470/2000, index 1469 evaluating
2022-01-04 16:25:26.770196 : 1480/2000, index 1479 evaluating
2022-01-04 16:26:29.611577 : 1490/2000, index 1489 evaluating
2022-01-04 16:27:30.929476 : 1500/2000, index 1499 evaluating
2022-01-04 16:28:30.932116 : 1510/2000, index 1509 evaluating
2022-01-04 16:29:30.870150 : 1520/2000, index 1519 evaluating
2022-01-

In [42]:
exsarsa_df = pd.DataFrame(values_exsar,columns=rl_columns)
exsarsa_df_iteration = exsarsa_df[iteration]
exsarsa_df_iteration.to_csv(URL + 'rl_csvs/exsarsa_df_iteration'+str(n)+'.csv',index=False)

In [43]:
exsarsa_df_final = pd.DataFrame(values_final_exsar,columns=rl_columns)
exsarsa_df_final.to_csv(URL + 'rl_csvs/exsarsa_df_final'+str(n)+'.csv',index=False)

In [44]:
exsarsa_df_iteration

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,0,56.324657,82.154716,49.0,49.0,53.650607,1099.235226,7.324657,33.154716,7.324657,33.154716,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
1,0,1,56.264008,78.358704,49.0,49.0,52.765806,861.933478,7.264008,29.358704,7.264008,29.358704,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
2,0,2,56.251286,56.684685,49.0,49.0,52.581142,59.054380,7.251286,7.684685,7.251286,7.684685,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
3,0,3,56.392979,82.467873,49.0,49.0,54.656134,1120.098498,7.392979,33.467873,7.392979,33.467873,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
4,0,4,56.425636,79.841614,49.0,49.0,55.140074,951.205140,7.425636,30.841614,7.425636,30.841614,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_VAR_VA..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,1999,195,111.810020,30.565590,124.0,124.0,148.595602,8729.988990,12.189980,93.434410,12.189980,93.434410,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399996,1999,196,110.611221,30.763823,124.0,124.0,179.259395,8692.984784,13.388779,93.236177,13.388779,93.236177,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399997,1999,197,109.997688,30.725050,124.0,124.0,196.064733,8700.216303,14.002312,93.274950,14.002312,93.274950,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."
399998,1999,198,110.784485,30.854906,124.0,124.0,174.649840,8676.008521,13.215515,93.145094,13.215515,93.145094,"['[""JOINᶲIRI_ID;...direct/P345;ᶲIRI_ID;...dire...","['[""JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direc..."


In [45]:
exsarsa_df_final

,index,iteration,pred_old,pred_new,real_old,real_new,mse_old,mse_new,mae_old,mae_new,rmse_old,rmse_new,old_tree,new_tree
0,0,199,56.531487,56.207592,49.0,49.0,56.723289,51.949383,7.531487,7.207592,7.531487,7.207592,"['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA...","['[""JOINᶲIRI_ID;...direct/P190;"", [""VAR_URI_VA..."
1,1,199,58.287270,59.852551,46.0,46.0,150.976994,191.893156,12.287270,13.852551,12.287270,13.852551,"['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2660;"", [""VAR_URI_V..."
2,2,199,60.374878,61.706520,48.0,48.0,153.137604,187.868693,12.374878,13.706520,12.374878,13.706520,"['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2053;"", [""VAR_URI_V..."
3,3,199,45.571594,45.571594,48.0,48.0,5.897155,5.897155,2.428406,2.428406,2.428406,2.428406,"['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']","['[""VAR_URI_URIᶲIRI_ID;...direct/P31;""]']"
4,4,199,111.163254,106.223946,49.0,49.0,3864.270121,3274.579952,62.163254,57.223946,62.163254,57.223946,"['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir...","['[""JOINᶲIRI_ID;...direct/P6284;ᶲIRI_ID;...dir..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,199,139.722412,138.682770,111.0,111.0,824.976957,766.335742,28.722412,27.682770,28.722412,27.682770,"['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P1725;"", [""VAR_URI_V..."
1996,1996,199,104.259911,59.035549,122.0,122.0,314.710773,3964.522069,17.740089,62.964451,17.740089,62.964451,"['[""LEFT_JOINᶲIRI_ID;...direct/P281;ᶲIRI_ID;.....","['[""JOIN"", [""VAR_URI_VARᶲIRI_ID;...direct/P17;..."
1997,1997,199,133.005463,74.937363,109.0,109.0,576.262237,1160.263262,24.005463,34.062637,24.005463,34.062637,"['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2632;"", [""VAR_VAR_V..."
1998,1998,199,156.947174,73.620453,136.0,136.0,438.784102,3891.207899,20.947174,62.379547,20.947174,62.379547,"['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_URI_V...","['[""JOINᶲIRI_ID;...direct/P2283;"", [""VAR_VAR_V..."


In [46]:
1+1

2

### Archivos enteros

In [47]:
import os
path = "/home/school/math/final_analysis.csv"

In [48]:
os.remove(URL + 'rl_csvs/ql_df.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/media/data/ccarmona/memoria/dataset/rl_csvs/ql_df.csv'

In [ ]:
os.remove(URL + 'rl_csvs/sarsa_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/neo_df.csv')

In [ ]:
os.remove(URL + 'rl_csvs/exsarsa_df.csv')

In [ ]:
ql_df.to_csv(URL + 'rl_csvs/ql_df.csv',index=False)

In [ ]:
sarsa_df.to_csv(URL + 'rl_csvs/sarsa_df.csv',index=False)

In [ ]:
neo_df.to_csv(URL + 'rl_csvs/neo_df.csv',index=False)

In [ ]:
exsarsa_df.to_csv(URL + 'rl_csvs/exsarsa_df.csv',index=False)